In [1]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [ ]:
#solid = film[i, j, k, 10][Si, SiF1, SiF2, SiF3, SiO SiO2, SiOF, SiOF2, SiO2F, SiO2F2]
#react_t g[F, O, ion] s  [1,          2,           3,          4,       5 ,   6,    7,    8,   9,  10]
#react_t g[F, O, ion] s  [Si,       SiF1,       SiF2,       SiF3,      SiO, SiO2, SiOF, SiOF2, SiO2F,SiO2F2]


film = np.zeros((50, 50, 100, 10))

bottom = 60
film[:, :, 0:bottom, 0] = 10 # bottom

height = 85

film[:, :15, bottom:height, 5] = 10
film[:, 35:, bottom:height, 5] = 10

slit = np.linspace(25, 0, 5)
for k in range(5):
    film[:, 15+k, bottom:int(slit[k]) + bottom, 5] = 10
    film[:, 30+k, bottom:int(slit[4-k]) + bottom, 5] = 10


In [ ]:
film = np.zeros((50, 50, 100, 10))

bottom = 70
film[:, :, 0:bottom, 0] = 10 # bottom
center = 25
height = 75

film[:, :20, bottom:height, 5] = 10
film[:, 30:, bottom:height, 5] = 10

In [2]:
# vertical mask
film = np.zeros((30, 100, 140, 10))

bottom = 100
height = 104

density = 10

radius = 30

center = 50

film[:, :45, bottom:height, 5] = density
film[:, 55:, bottom:height, 5] = density
# film[:, :, 0:bottom, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
# film[:, :, 0:bottom, 1] = 0 # bottom
# film[:, :, 0:bottom, 2] = 0 # bottom

In [3]:
sumFilm = np.sum(film, axis=-1)

print(sumFilm.shape)

(30, 100, 140)


In [4]:
# film = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(film[:, :, :, 5]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:58006/index.html?ui=P_0x2328b3d2870_0&reconnect=auto" class="pyvis…

In [5]:
etchfilm = film

In [6]:
import Multi_Species_SF_etching

In [7]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_SF_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                                         depoThick=120, center_with_direction=np.array([[etchfilm.shape[0]/2,center,75]]), 
                                         range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]]), InOrOut=[1], yield_hist=np.array([None]),
                                        reaction_type=False, param = [1.6, -0.7], N = 300000, 
                                        sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                                        celllength=1e-5, kdtreeN=5, tstep=1e-5,
                                        substrateTop=40,posGeneratorType='top', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [8]:
T = 300
Cm = (2*1.380649e-23*T/(27*1.66e-27) )**0.5 # (2kT/m)**0.5 27 for the Al

def max_velocity_u( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.cos(2*np.pi*random2))

def max_velocity_w( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.sin(2*np.pi*random2))

def max_velocity_v( random3):
    return -Cm*np.sqrt(-np.log(random3))

In [9]:
# O2

N = int(1e7)
velosity_matrix = np.zeros((N, 3))
# tstep=1e-5
# celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# ion_ration = int(N/8)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

[[-0.77519202 -0.57784155 -0.25529685  0.        ]
 [-0.21557954 -0.75214459 -0.6227391   0.        ]
 [-0.39186095 -0.28877238 -0.87353048  0.        ]
 [ 0.08714513 -0.57884836 -0.81076526  0.        ]
 [ 0.11746587  0.42217712 -0.89887054  0.        ]
 [ 0.60381169 -0.06628451 -0.7943663   0.        ]
 [-0.09299176 -0.71424321 -0.69369242  0.        ]
 [-0.82830567 -0.15071843 -0.53962364  0.        ]
 [ 0.02650076 -0.2194957  -0.97525348  0.        ]
 [ 0.41240648 -0.72981405 -0.54524521  0.        ]]


In [ ]:
# F+ Ar+ 
N = int(1e7)
velosity_matrix = np.zeros((N, 3))

typeID = np.zeros(N)

ion_ration = int(N/8)
typeID[-ion_ration:] = 2
velosity_matrix[:, 0] = np.random.rand(N)*0.001
velosity_matrix[:, 1] = np.random.rand(N)*0.001
velosity_matrix[:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
# step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(4e4), int(4e6), int(12e5),2, 10, 100)
etching = testEtch.inputParticle(125,vel_type_shuffle,int(4e3),int(1e7), 5)

running: 127B [05:38, 2.66s/B]                                                                      


KeyboardInterrupt: 

In [ ]:
etching = testEtch.inputParticle(125,vel_type_shuffle[:,:3], vel_type_shuffle[:,-1], 4e-2, 5)

In [13]:
sumFilm = np.sum(etchfilm, axis=-1)

depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :,1]!=0, etchfilm[:, :, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etchfilm[:, :, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etchfilm[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:58006/index.html?ui=P_0x232c3952b40_1&reconnect=auto" class="pyvis…

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:25, :, :,1]!=0, etching[0][:25, :, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:25, :, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:25, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def removeFloat(film):  # fast scanZ
    
    # 获取当前平面的非零元素布尔索引
    current_plane = film != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(film, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= film[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= film[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= film[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= film[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= film[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= film[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition] = 0
    
    return film


In [ ]:
testRemove = removeFloat(sumFilm)

In [ ]:
depo1 = torch.Tensor(testRemove != 0).to_sparse()
depo1 = depo1.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(etching[0][:, :25, :,1]!=0, etching[0][:, :25, :,2]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

depo2 = torch.Tensor(etching[0][:, :25, :,5]!=0).to_sparse()
depo2 = depo2.indices().numpy().T

substrute = torch.Tensor(etching[0][:, :25, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
depomesh2 = pv.PolyData(depo2)
depomesh2["radius"] = np.ones(depo2.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed2 = depomesh2.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='red')
p.add_mesh(depoglyphed2, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)
# np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)
depo1 = torch.Tensor(np.logical_and(etching[0][:, :, :, 5]>0, sumFilm[:, :, :,]>0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(etching[0][:, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :]==5).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
pointcloud = testEtch.get_pointcloud(sumFilm)

In [ ]:
point_cloud = pv.PolyData(pointcloud[:, 3:])
vectors = pointcloud[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=5,
)

# Display the arrowscyan

# sphere = pv.Sphere(radius=4, center=(21, 75, 114.5))
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
depo1 = torch.Tensor(pointcloud[:,3:]).to_sparse()
depo1 = depo1.indices().numpy().T


depomesh = pv.PolyData(pointcloud[:,3:])
depomesh["radius"] = np.ones(pointcloud[:,3:].shape[0])*0.5
geom = pv.Box()


# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(etching[0], axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, 50:, :,]!=10, sumFilm[:, 50:, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, 50:, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
sumFilm = np.sum(testEtch.film, axis=-1)

depo1 = torch.Tensor(np.logical_and(sumFilm[:, :, :,]!=10, sumFilm[:, :, :,]!=0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(sumFilm[:, :, :,]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
testEtch.parcel.shape

In [ ]:
testEtch.parcel[106]

In [ ]:
testEtch.parcel[100, :3] += testEtch.parcel[100, 3:6] 
print(testEtch.parcel[100])

In [ ]:
testEtch.parcel[10000:11000][:,8].astype(int)
